### Import Libraries

In [1]:
import pandas as pd

### Load Raw Data

In [2]:
employee_survey_data = pd.read_csv('../data/employee_survey_data.csv')

general_data = pd.read_csv('../data/general_data.csv')

manager_survey_data = pd.read_csv('../data/manager_survey_data.csv')

in_time = pd.read_csv('../data/in_time.csv').rename(columns={'Unnamed: 0': 'EmployeeID'}).set_index('EmployeeID')

out_time = pd.read_csv('../data/out_time.csv').rename(columns={'Unnamed: 0': 'EmployeeID'}).set_index('EmployeeID')


In [4]:
in_time = in_time.apply(pd.to_datetime)

out_time = out_time.apply(pd.to_datetime)

actual_working_hours = out_time - in_time

working_hours_float = actual_working_hours.apply(lambda x: x.dt.total_seconds() / 3600)

mean_hours = working_hours_float.mean(axis=1)

overtime_days = (working_hours_float > 8).sum(axis=1)

absent_days = in_time.isnull().sum(axis=1)

time_features = pd.DataFrame({
    'EmployeeID': mean_hours.index,
    'Mean_Work_Hours': mean_hours.values,
    'Overtime_Days_Count': overtime_days.values,
    'Absent_Days_Count': absent_days.values
})

In [19]:
df_final = general_data.merge(employee_survey_data,on='EmployeeID', how='left')
df_final = df_final.merge(manager_survey_data, on = 'EmployeeID', how='left')
df_final = df_final.merge(time_features, on ='EmployeeID', how='left')

In [24]:
df_final.head(3)

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Mean_Work_Hours,Overtime_Days_Count,Absent_Days_Count
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,0,0,3.0,4.0,2.0,3,3,7.373651,0,29
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,1,4,3.0,2.0,4.0,2,4,7.718969,42,25
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,0,3,2.0,2.0,1.0,3,3,7.013240,0,19


In [23]:
df_final.isnull().sum().loc[lambda x:x>0]

NumCompaniesWorked         19
TotalWorkingYears           9
EnvironmentSatisfaction    25
JobSatisfaction            20
WorkLifeBalance            38
dtype: int64